## StarTracker Position comparisons - 09-Feb-23

Craig Lage - comparing header values with EFD

In [ ]:
import sys, time, os, asyncio, glob
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as pf
from astropy.time import Time, TimeDelta
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5, SkyCoord
import astropy.units as u
from lsst.obs.lsst.translators.latiss import AUXTEL_LOCATION
from lsst.obs.lsst.translators.lsst import SIMONYI_LOCATION

import lsst.afw.image as afwImage
from lsst.summit.utils.utils import starTrackerFileToExposure
from lsst.summit.utils.astrometry.anet import CommandLineSolver
from lsst.summit.utils.astrometry.utils import runCharactierizeImage
from lsst_efd_client import EfdClient

In [ ]:
solver = CommandLineSolver('/project/shared/ref_cats/astrometry_net/')

In [ ]:
client = EfdClient('summit_efd')

In [ ]:
cameras = [['Wide', 101], ['Narrow', 102]]
dates = [20230221]
for [camera, num] in cameras:
    for date in dates:
        year = int(date/10000)
        month = int((date - 10000 * year)/100)
        day = int((date - 10000 * year - 100 * month))
        path = f"/project/GenericCamera/{num}/{year}/{month:02}/{day:02}/"
        #outfile = open(f"/scratch/cslage/starTracker/StarTracker_Headers_Mount_{camera}_{date}.txt", "w")
        #outfile.write("SeqNum\tRA(header)\tDec(header)\tRA(MTPtg)\tDec(MTPtg)\tAz(header)\tEl(header)\tAz(MTMount)\tEl(MTMount)\tAz(MTMount)\tEl(MTMount)\n")
        for seqNum in range(3,4):
            try:    
                filename = path + f"GC{num}_O_{date}_{seqNum:06}.fits"
                exp = starTrackerFileToExposure(filename)
                hdulist = pf.open(filename)
                header = hdulist[0].header
                raStart = header["RASTART"]
                decStart = header["DECSTART"]
                azHdr = (header["AZSTART"]+header["AZEND"])/2.0
                azStart = azStart % 360
                elHdr = (header["ELSTART"]+header["ELEND"])/2.0
                start_time = Time(header["DATE-OBS"], scale='tai')

            except:
                print(f"{camera}, {date}_{seqNum} Failed!")
                continue
            end_time = start_time + TimeDelta(1.0, format='sec')
            azMTM = await client.select_time_series('lsst.sal.MTMount.azimuth', \
                                            ['actualPosition'],  start_time.utc, end_time.utc)
            elMTM = await client.select_time_series('lsst.sal.MTMount.elevation', \
                                            ['actualPosition'],  start_time.utc, end_time.utc) 
            azMTMount = azMTM.values[:,0][0]
            azMTMount = azMTMount % 360
            elMTMount = elMTM.values[:,0][0]
            azPtg = await client.select_time_series('lsst.sal.MTPtg.mountPosition', \
                                            ['*'],  start_time.utc, end_time.utc)
            azMTPtg = azPtg['azimuthActualPosition'].values[0]
            azMTPtg = azMTPtg % 360
            elMTPtg = azPtg['elevationActualPosition'].values[0]
            raMTPtg = azPtg['ra'].values[0]
            decMTPtg = azPtg['declination'].values[0]
            #outfile.write(f"{seqNum}\t{raStart:.5f}\t{decStart:.5f}\t{raMTPtg:.5f}\t{decMTPtg:.5f}\t{azStart:.5f}\t{elStart:.5f}\t{azMTMount:.5f}\t{elMTMount:.5f}\t{azMTPtg:.5f}\t{elMTPtg:.5f}\n")
        
            print(f"{camera}, {date}_{seqNum}")
            print(f"AzHdr = {azHdr:.5f}, AzMTMount = {azMTMount:.5f}, AzMTPtg = {azMTPtg:.5f}")
            print(f"ElHdr = {elHdr:.5f}, ElMTMount = {elMTMount:.5f}, ElMTPtg = {elMTPtg:.5f}")
            print(f"RAHdr = {raStart:.5f}, RAMTPtg = {raMTPtg:.5f}")
            print(f"DecHdr = {decStart:.5f}, DecMTPtg = {decMTPtg:.5f}")
        #outfile.close()
    